In [4]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
from bandbLIMID import BranchAndBoundLIMIDInference
from exemple_robot import createIDRobot
import numpy as np

maze=["---------",
      "--     --",
      "-  - -  -",
      "-- - - --",
      "-  - - $-",
      "--     --",
      "---------"]
nbStage=2
xInitial=3
yInitial=2
ID=createIDRobot(nbStage,xInitial,yInitial,maze)
#gnb.showInfluenceDiagram(ID)
ordre=[]
for i in range(nbStage):
    ordre.append(ID.idFromName("d_"+str(i)))

def checkCPTs(bn):
    nodes=[]
    for i in bn.nodes():
        if bn.isChanceNode(i):
            p=bn.cpt(i).margSumOut(bn.variable(i).name())
            if (abs(p.min()-1)>1e-10) or (abs(p.max()-1)>1e-10):
                print(f"erreur  CPT for {bn.variable(i).name()} :")
                #print(bn.cpt(i))
                #nodes.append(bn.variable(i).name())
                nodes.append(i)
    if len(nodes)==0:
            print("pas d'erreur")
    return nodes
nodes=checkCPTs(ID)
bnb=BranchAndBoundLIMIDInference(ID,ordre)



pas d'erreur


In [5]:
bnb.setVerbose(True)
bnb.branchAndBound()

####################################################
####################################################################################################################
####################################################################################################################
####################################################################################################################
Nouvelle Branche, nombre de branche au total: 20 nombre de branches coupées: 0
On va développer le noeud d_0 id 25 On va donc chercher les parents du noeud d_1
Le domaine de d_0 : ['0', '1', '2', '3', '4'] ,les valeurs déjà processed: ['0', '1', '2', '3'] ce qu'on va process : 4
d_0 de contexte :
ns_0 : 0
es_0 : 0
ss_0 : 1
ws_0 : 1
d_0 : 4
Les parents de  d_1  sont :  ['ns_1', 'es_1', 'ss_1', 'ws_1']
ajout d'une couche, de racine, ns_1 avec les noeuds de décisions crées étants : ['d_1', 'd_1', 'd_1', 'd_1', 'd_1', 'd_1', 'd_1', 'd_1', 'd_1', 'd_1', 'd_1', 'd_1', 'd_1', 'd_1', 'd_1', 'd_1']

In [12]:
andOrGraph=bnb.andOrGraph
def dir(d):
    if d==0: 
        return "gauche"
    if d==1: 
        return "haut"
    if d==2: 
        return "droite"
    if d==3: 
        return "bas"
    if d==4: 
        return "bouge pas"    
print(f"nombre de noeux chance : {len(andOrGraph.getNoeudChance())}")
print(f"nombre de noeux décision: {len(andOrGraph.getNoeudDecision())}")
for node in andOrGraph.getNoeud():
    if(ID.variable(node.getNodeID()).isDecisionNode()):
        print(node.getDecisionOptimale())
"""decisiond0=np.zeros((5,1))
decisiond1=np.zeros((5,1))
for node in bnb.andOrGraph.getNoeud():
    if(ID.variable(node.getNodeID()).name()=="d_0"):
        if not node.getDecisionOptimale()==None:
            decisiond0[node.getDecisionOptimale()]+=1
    if(ID.variable(node.getNodeID()).name()=="d_1"):
        if not node.getDecisionOptimale()==None:
            decisiond1[node.getDecisionOptimale()]+=1"""

            #relaxe(relaxe))=relaxe
            #resultats expérimentaux

nombre de noeux chance : 1782
nombre de noeux décision: 1296


AttributeError: 'DiscreteVariable' object has no attribute 'isDecisionNode'

In [1]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
from bandbLIMID import BranchAndBoundLIMIDInference
from exemple_robot import createLIMIDRobot
import numpy as np

maze=["---------",
      "--     --",
      "-  - -  -",
      "-- - - --",
      "-  - - $-",
      "--     --",
      "---------"]
nbStage=2
xInitial=3
yInitial=2
ID=createLIMIDRobot(nbStage,xInitial,yInitial,maze)
#gnb.showInfluenceDiagram(ID)
ordre=[]
for i in range(nbStage):
    ordre.append(ID.idFromName("d_"+str(i)))

def checkCPTs(bn):
    nodes=[]
    for i in bn.nodes():
        if bn.isChanceNode(i):
            p=bn.cpt(i).margSumOut(bn.variable(i).name())
            if (abs(p.min()-1)>1e-10) or (abs(p.max()-1)>1e-10):
                print(f"erreur  CPT for {bn.variable(i).name()} :")
                #print(bn.cpt(i))
                #nodes.append(bn.variable(i).name())
                nodes.append(i)
    if len(nodes)==0:
            print("pas d'erreur")
    return nodes
nodes=checkCPTs(ID)
bnb=BranchAndBoundLIMIDInference(ID,ordre,verbose=True)
bnb.branchAndBound()


pas d'erreur
ajout d'une couche, de racine, ns_0 avec les noeuds de décisions crées étants : ['d_0', 'd_0', 'd_0', 'd_0', 'd_0', 'd_0', 'd_0', 'd_0', 'd_0', 'd_0', 'd_0', 'd_0', 'd_0', 'd_0', 'd_0', 'd_0'] de taille 16
Calcul des bornes pour les noeuds de décisions
####################################################################################################################
####################################################################################################################
####################################################################################################################
####################################################################################################################
Nouvelle Branche, nombre de branche au total: 1 nombre de branches coupées: 0
On va développer le noeud d_0 id 22 On va donc chercher les parents du noeud d_1
Le domaine de d_0 : ['0', '1', '2', '3', '4'] ,les valeurs déjà processed: [] ce qu'on va process : 0
d_0 d

In [4]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
from bandbLIMID import BranchAndBoundLIMIDInference
from exemple_robot import createRandomID
import numpy as np

gnb.showInfluenceDiagram(createRandomID(3,10,2,9))

InvalidDirectedCycle: [pyAgrum] Directed cycle detected: Add a mono-cycle in a dag !